# Сверточные нейронные сети для анализа текста

## Импорты

In [1]:
import re
import gensim
import numpy as np
import pandas as pd
from tqdm import tqdm
from string import punctuation
from functools import lru_cache

from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer

import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.callbacks import EarlyStopping

nltk.download("punkt")
tqdm.pandas()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\avpat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Настройки

In [2]:
MAX_WORDS = 5000
MAX_LEN = 30
NUM_CLASSES = 5

EPOCHS = 20
BATCH_SIZE = 64

MODEL_PATH = './data/ruwikiruscorpora_upos_cbow_300_10_2021/'

## Загрузка данных

In [3]:
data = pd.read_excel('./data/отзывы за лето.xls')
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


## Предобработка текста

In [4]:
stop_words = set(get_stop_words('ru')) - {'не', 'ни', 'нет'} # оставим отрицания
exclude = set(punctuation) - set('=:)]([') # Оставим смайлы
morpher = MorphAnalyzer()

@lru_cache(None)
def lemmatize(word):
    return morpher.parse(word)[0].normal_form

def preprocess_text(text):
    text = str(text)
    text.lower()
    text = re.sub(r'(не|ни|нет)\s', 'не', text)

    for char in exclude:
        text = text.replace(char, ' ')

    words = [word for word in text.split() if word not in stop_words]
    words = [word for word in words if len(word) >= 2]
    words = [lemmatize(word) for word in words]
    return ' '.join(words)

In [5]:
data['preprocessed_text'] = data['Content'].progress_apply(preprocess_text)
data = data[data['preprocessed_text'] != '']
data.sample(10)

100%|██████████| 20659/20659 [00:01<00:00, 14081.61it/s]


,Rating,Content,Date,preprocessed_text
7788,5,Ауе,2017-08-03,ауе
373,5,Отличный программы,2017-08-13,отличный программа
1056,5,От лично,2017-08-12,от лично
10320,4,При в ходе приходится повторно в водить код.,2017-07-30,при ход приходиться повторно водить код
13148,5,Очень удобно!все операции в телефоне. Ну кроме...,2017-07-26,очень удобно операция телефон ну снятия:)
11179,4,"Ну наконец-то, спустя несколько месяцев и милл...",2017-07-29,ну спустя месяц негативный отзыв приложение пр...
5885,5,Удобное приложение,2017-08-05,удобный приложение
11097,1,В один прекрасный момент приложение зависло. Н...,2017-07-29,прекрасный момент приложение зависнуть ничего ...
7778,4,Удобно спасибо,2017-08-03,удобно
19998,5,Удобно,2017-06-17,удобно


In [6]:
train_corpus = ' '.join(data['preprocessed_text'])
tokens = word_tokenize(train_corpus)
tokens_filtered = [word for word in tokens if word.isalnum()]
len(tokens_filtered)

115671

In [7]:
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(MAX_WORDS-1)]
tokens_filtered_top[:5]

['приложение', 'удобно', 'всё', 'очень', 'работать']

In [8]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [9]:
data_train = np.asarray([text_to_sequence(text, MAX_LEN) for text in data['preprocessed_text']], dtype=np.int32)
X_train, X_val, y_train, y_val = train_test_split(data_train, data['Rating'], test_size=0.25)

le = LabelEncoder()
y_train = le.fit_transform(y_train) 
y_val = le.transform(y_val)

## Учим conv сеть для классификации

In [10]:
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_val = keras.utils.to_categorical(y_val, NUM_CLASSES)

def get_model(max_words, max_len, num_classes, embeddings_initializer=None):
    model = Sequential()
    model.add(Embedding(input_dim=max_words,
                        output_dim=128,
                        input_length=max_len,
                        embeddings_initializer=embeddings_initializer))

    model.add(Conv1D(128, 3))
    model.add(Activation("relu"))
    model.add(GlobalMaxPool1D())
    model.add(Dense(10))
    model.add(Activation("relu"))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

## Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/

In [11]:
ruwikiruscorpora = gensim.models.KeyedVectors.load_word2vec_format(MODEL_PATH + 'model.bin', binary=True)
ruwikiruscorpora_matrix = [ruwikiruscorpora[i][:128] for i in range(MAX_WORDS)]
embeddings_initializer = keras.initializers.Constant(ruwikiruscorpora_matrix)

In [12]:
early_stopping=EarlyStopping(monitor='val_loss')

model = get_model(MAX_WORDS, MAX_LEN, NUM_CLASSES, embeddings_initializer)
history = model.fit(X_train, y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
218/218 [==============================] - 2s 10ms/step - loss: 0.9841 - accuracy: 0.6965 - val_loss: 0.9958 - val_accuracy: 0.6955
Epoch 2/20
218/218 [==============================] - 2s 10ms/step - loss: 0.7573 - accuracy: 0.7330 - val_loss: 0.8126 - val_accuracy: 0.7162


In [13]:
score = model.evaluate(X_val, y_val, batch_size=BATCH_SIZE, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

81/81 [==============================] - 0s 2ms/step - loss: 0.7386 - accuracy: 0.7378


Test score: 0.7385953068733215
Test accuracy: 0.7377812266349792


## Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

In [14]:
early_stopping=EarlyStopping(monitor='val_loss')

model = get_model(MAX_WORDS, MAX_LEN, NUM_CLASSES)
history = model.fit(X_train, y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
218/218 [==============================] - 2s 10ms/step - loss: 0.9021 - accuracy: 0.7411 - val_loss: 0.6962 - val_accuracy: 0.7576
Epoch 2/20
218/218 [==============================] - 2s 10ms/step - loss: 0.6011 - accuracy: 0.7862 - val_loss: 0.6871 - val_accuracy: 0.7589


In [15]:
score = model.evaluate(X_val, y_val, batch_size=BATCH_SIZE, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

81/81 [==============================] - 0s 2ms/step - loss: 0.6347 - accuracy: 0.7822


Test score: 0.6346622109413147
Test accuracy: 0.782195508480072


Модель с предобученными весами показала результат хуже, чем модель с весами инициализированными по умолчанию.